# 1. Script details

Name of script: OpenDataAPIQuery <br>
Description:  Using Python to query the NHSBSA open data portal API. <br>

# 2. Load packages

List packages we will use

In [1]:
import grequests
import pandas as pd
import re
import requests
import warnings
import urllib.parse

warnings.simplefilter("ignore", category=UserWarning)

Install packages if they aren't already using `Pip/Conda install -r requirements.txt`

# 3. Define variablesDefine the url for the API call

Define the url for the API call

In [2]:
base_endpoint = 'https://opendata.nhsbsa.net/api/3/action/'
package_list_method = 'package_list'     # List of data-sets in the portal
package_show_method = 'package_show?id=' # List all resources of a data-set
action_method = 'datastore_search_sql?'  # SQL action method

Send API call to get list of data-sets

In [3]:
datasets_response = requests.get(base_endpoint +  package_list_method).json()

Now lets have a look at the data-sets currently available

In [4]:
print(datasets_response['result'])

['23362', '25521', 'bnf-code-information-current-year', 'bnf-code-information-historic', 'bnf-code-information-monthly-changes', 'consolidated-pharmaceutical-list', 'contractor-details', 'dental-activity-delivered-by-newly-qualified-foundation-dentists', 'dental-new-patient-premium', 'dispensing-practice-name-and-address', 'english-community-pharmacy-covid-19-vaccination-fees-current', 'english-community-pharmacy-covid-19-vaccination-service-fees-historic', 'english-contractor-monthly-general-dental-activity', 'english-contractor-monthly-general-dental-and-orthodontic-contractual-dataset', 'english-contractor-monthly-orthodontic-activity', 'english-prescribing-data-epd', 'english-prescribing-dataset-epd-with-snomed-code', 'finalised-secondary-care-medicines-data-scmd-with-indicative-price', 'foi-01003', 'foi-01004', 'foi-01005', 'foi-01006', 'foi-01008', 'foi-01009', 'foi-01012', 'foi-01016', 'foi-01017', 'foi-01019', 'foi-01020', 'foi-01021', 'foi-01022', 'foi-01024', 'foi-01025', 'fo

For this example we're interested in the English Prescribing Dataset (EPD).
We know the name of this data-set so can set this manually, or access it 
from datasets_response.

In [5]:
dataset_id = "english-prescribing-data-epd"

# 4. API calls for single month

Define the parameters for the SQL query

In [6]:
resource_name = 'EPD_202001' # For EPD resources are named EPD_YYYYMM
pco_code = '13T00' # Newcastle Gateshead CCG
bnf_chemical_substance = '0407010H0' # Paracetamol

Build SQL query (WHERE criteria should be enclosed in single quotes)

In [7]:
single_month_query = "SELECT * " \
                     f"FROM `{resource_name}` " \
                     f"WHERE pco_code = '{pco_code}' " \
                     f"AND bnf_chemical_substance = '{bnf_chemical_substance}'"

Build API call

In [8]:
single_month_api_call = f"{base_endpoint}" \
                        f"{action_method}" \
                        "resource_id=" \
                        f"{resource_name}" \
                        "&" \
                        "sql=" \
                        f"{urllib.parse.quote(single_month_query)}" # Encode spaces in the url

Grab the response JSON as a list

In [10]:
single_month_response = requests.get(single_month_api_call).json()

Extract records in the response to a dataframe

In [11]:
single_month_df  = pd.json_normalize(single_month_response['result']['result']['records'])

Lets have a quick look at the data

In [12]:
single_month_df.head()

,YEAR_MONTH,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,AREA_TEAM_NAME,AREA_TEAM_CODE,PCO_NAME,PCO_CODE,PRACTICE_NAME,PRACTICE_CODE,ADDRESS_1,...,BNF_CODE,BNF_DESCRIPTION,BNF_CHAPTER_PLUS_CODE,QUANTITY,ITEMS,TOTAL_QUANTITY,ADQUSAGE,NIC,ACTUAL_COST,UNIDENTIFIED
0,202001,NORTH OF ENGLAND,Y54,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Q49,NEWCASTLE GATESHEAD CCG,13T00,NEWCASTLE GP IN MOLINEUX WIC,Y05671,MOLINEUX WALK-IN CENTRE,...,0407010H0AAA7A7,Paracetamol 120mg/5ml oral solution paediatric...,04: Central Nervous System,100.0,1,100.0,0.80000,1.19,1.21887,False
1,202001,NORTH OF ENGLAND,Y54,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Q49,NEWCASTLE GATESHEAD CCG,13T00,PONTELAND RD WIC,Y03375,PONTELAND RD WIC,...,0407010H0AAACAC,Paracetamol 250mg/5ml oral suspension,04: Central Nervous System,200.0,1,200.0,3.33333,4.16,3.88041,False
2,202001,NORTH OF ENGLAND,Y54,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Q49,NEWCASTLE GATESHEAD CCG,13T00,WALK-IN CENTRE GP IN-HOURS,Y04106,WALK-IN CENTRE,...,0407010H0AAA7A7,Paracetamol 120mg/5ml oral solution paediatric...,04: Central Nervous System,100.0,1,100.0,0.80000,1.19,1.21887,False
3,202001,NORTH OF ENGLAND,Y54,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Q49,NEWCASTLE GATESHEAD CCG,13T00,WALK-IN CENTRE GP IN-HOURS,Y04106,WALK-IN CENTRE,...,0407010H0AABGBG,Paracetamol 250mg/5ml oral suspension sugar free,04: Central Nervous System,80.0,1,80.0,1.33333,1.07,1.10730,False
4,202001,NORTH OF ENGLAND,Y54,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Q49,NEWCASTLE GATESHEAD CCG,13T00,ST OSWALDS PALLIATIVE CARE,Y00505,ST OSWALDS HOSPICE,...,0407010H0AAACAC,Paracetamol 250mg/5ml oral suspension,04: Central Nervous System,500.0,1,500.0,8.33333,10.40,9.68241,False


You can use any of the fields listed in the data-set within the SQL query as 
part of the select or in the where clause in order to filter.

Information on the fields present in a data-set and an accompanying data 
dictionary can be found on the page for the relevant data-set on the Open Data 
Portal.

# 5. API calls for data for multiple months

Now that you have extracted data for a single month, you may want to get the 
data for several months, or a whole year.

Firstly we need to get a list of all of the names and resource IDs for every 
EPD file. We therefore extract the metadata for the EPD dataset.

In [13]:
metadata_repsonse  = requests.get(f"{base_endpoint}" \
                                  f"{package_show_method}" \
                                  f"{dataset_id}").json()

Resource names and IDs are kept within the resources table returned from the 
package_show_method call.

In [14]:
resources_table  = pd.json_normalize(metadata_repsonse['result']['resources'])

We only want data for one calendar year, to do this we need to look at the 
name of the data-set to identify the year. For this example we're looking at 
2020.

In [15]:
resource_name_list = resources_table[resources_table['name'].str.contains('2020')]['name']

## 5.1. For loop

We can do this with a for loop that makes all of the individual API calls for 
you and combines the data together into one dataframe

Initialise dataframe that data will be saved to

In [16]:
for_loop_df = pd.DataFrame()

As each individual month of EPD data is so large it will be unlikely that your 
local system will have enough RAM to hold a full year's worth of data in 
memory. Therefore we will only look at a single CCG and chemical substance as 
we did previously

Loop through resource_name_list and make call to API to extract data, then 
bind each month together to make a single data-set

Lets have a quick look at the data

In [ ]:
for_loop_df.head()

## 5.2. Async

We can call the API asynchronously and this will result in an approx 10x speed 
increase over a for loop for large resource_names by vectorising our approach.

Construct the SQL query as a function

In [ ]:
def async_query(resource_name):
    query = "SELECT * " \
            f"FROM `{resource_name}` " \
            f"WHERE pco_code = '{pco_code}' " \
            f"AND bnf_chemical_substance = '{bnf_chemical_substance}'"
    return(query)

Create the API calls

In [ ]:
async_api_calls = []
for x in resource_name_list:
    async_api_calls.append(
        f"{base_endpoint}" \
        f"{action_method}" \
        "resource_id=" \
        f"{x}" \
        "&" \
        "sql=" \
        f"{urllib.parse.quote(async_query(x))}" # Encode spaces in the url 
    )

Use grequests to get the results

In [ ]:
dd = (grequests.get(u) for u in async_api_calls)
res = grequests.map(dd)

Check that everything is a success

In [ ]:
for x in res:
    if x.ok:
        print(True)
    else:
        print(False)

Parse the output into a list of dataframes and concatenate the results

In [ ]:
async_df = pd.DataFrame()

for x in res:
    # Grab the response JSON as a temporary list
    tmp_response = x.json()
    
    # Extract records in the response to a temporary dataframe
    tmp_df = pd.json_normalize(tmp_response['result']['result']['records'])
    
    # Bind the temporary data to the main dataframe
    async_df = async_df.append(tmp_df)

Lets have a quick look at the data

In [ ]:
async_df.head()

# 6. Export the data

In [ ]:
single_month_df.to_csv('single_month.csv')
for_loop_df.to_csv('for_loop.csv')
async_df.to_csv('async.csv')

In [2]:
pip install grequests

     ---------------------------------------- 0.0/45.6 kB ? eta -:--:--
     -------- ------------------------------- 10.2/45.6 kB ? eta -:--:--
     ------------------------- ------------ 30.7/45.6 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 45.6/45.6 kB 250.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.6 MB 991.0 kB/s eta 0:00:02
   - -------------------------------------- 0.0/1.6 MB 991.0 kB/s eta 0:00:02
   - -------------------------------------- 0.1/1.6 MB 409.6 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/1.6 MB 512.0 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/1.6 MB 504.4 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.6 MB 502.3 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.6 MB 513.4 kB/s eta 0:00:03
   ------- -------------------------------- 0.3/1.6 MB 681.0 kB/s eta 0:00:02
  

DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Program Files\\Python311\\Include\\greenlet'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
